# Exclusions for segmentation analysis
### 11/6/20 

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import os
import glob
import math
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
from random import randrange
from sklearn.metrics import jaccard_score
import random
%autosave 5

Autosaving every 5 seconds


# I. Import data

In [31]:
#if imac == 0:
#    filename = '/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/analysis/parsing_log_file/pickle/data_dict_pilot_b1_b2_b3_b4_b5_b6_b7_b8.p'
#    #filename = '/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/analysis/parsing_log_file/pickle/data_dict_b6.p'
#elif imac == 1:
#    filename = '/Users/alexreblando/Library/Mobile Documents/com~apple~CloudDocs/Documents/GitHub/data_analysis_ebs/online_experiment/analysis/parsing_log_file/pickle/data_dict_pilots_b1_b2_b3_b4_b5_b6_b7.p'
filename = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/parsing_log_file/pickle/data_dict_pilot_b1_b2_b3_b4_b5_b6_b7_b8_newlaptop.p'   
data_dict = pickle.load( open( filename, "rb" ) )
og_n = len(data_dict)

In [3]:
len(data_dict.keys())

391

In [67]:
data_dict['/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822']

KeyError: '/Users/alexreblando/Documents/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822'

In [143]:
dd = copy.deepcopy(data_dict)

In [144]:


for key in dd:
    #print(key)
    if dd[key]['prolific_id'].iloc[0] == '5f6aa61123368b1893fd12d6':
        print(key)


/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/160_1_online_exp_2020-10-20_12h03.36.343


IndexError: single positional indexer is out-of-bounds

In [145]:
dd['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/6_1_online_exp_2020-10-20_10h04.26.275']

,story_presses,A1,A2,A3,A4,A5,A6,A7,A8,prolific_id,story,p_type,persp,alt_persp,sa_order


# II. Story files
## A. import story files

In [52]:
directory_stories = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/1:8:2020_edited_stories'


filenames_stories = glob.glob(directory_stories + '/*.xlsx')

dfs_stories = []
    
for filename in filenames_stories:
    dfs_stories.append(pd.read_excel(filename))

## B. organize putative event boundaries

In [4]:
#mark the boundaries with a '1'
#output: story_boundaries, where the first column has the location boundaries and the second column has the social
#boundaries

story_boundaries = dict()

#read in the location and social event values from the story files 
for s in range(16):
    this_story = int(dfs_stories[s]['story'].iloc[0])
    keys2 = dfs_stories[s]['locationEvent'].values
    keys3 = dfs_stories[s]['socialEvent'].values
    story_boundaries[this_story] = keys2[:, np.newaxis]
    story_boundaries[this_story] = np.concatenate((story_boundaries[this_story], keys3[:, np.newaxis]), axis = 1)
    
#mark the changes or boundaries from one event to another with a '1' and delete first two columns
for key in story_boundaries:
    location = story_boundaries[key][:,0]
    social = story_boundaries[key][:,1]
    for i in range(0, len(location)):  
        if location[i] > location[i-1]:
            location[i] = 7
    for i in range(0, len(location)):  
        if location[i] != 7:
            location[i] = 0
    for i in range(0, len(social)):
        if social[i] > social[i - 1]:
            social[i] = 7
    for i in range(0, len(social)):
        if social[i] != 7:
            social[i] = 0
    location[0] = 7
    social[0] = 7
    location2 = location > 1
    social2 = social > 1
    location2 = location2.astype(int)
    social2 = social2.astype(int)
    story_boundaries[key] = np.concatenate((story_boundaries[key], location2[:, np.newaxis]), axis = 1) 
    story_boundaries[key] = np.concatenate((story_boundaries[key], social2[:, np.newaxis]), axis = 1)
    story_boundaries[key] = np.delete(story_boundaries[key] ,np.s_[0:2],axis=1)

# III. Exclusions
## A. Exclusion of participants who only pressed '1' or '9' (not including first row 10/14)

In [148]:
keys_to_del = []
for key in data_dict:
    kp = data_dict[key]['story_presses']
    kp_i = kp.iloc[1:]
    check_kp_i = len(set(kp_i))
    if check_kp_i == 1:
        #print(key)
        #print(data_dict[key]['prolific_id'].iloc[0])
        keys_to_del.append(key)
        
for i in range(0, len(keys_to_del)):
    print(keys_to_del[i])
    del data_dict[keys_to_del[i]]
    
e1 = len(keys_to_del)

/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot2/8/PARTICIPANT_1_online_exp_2020-05-11_12h52.50.131
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot3/4/PARTICIPANT_1_online_exp_2020-05-20_17h03.06.828
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/23_1_online_exp_2020-06-09_13h39.42.955
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/145_1_online_exp_2020-06-10_15h14.14.911
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/171_1_online_exp_2020-06-10_21h32.27.534
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/33_1_online_exp_2020-06-10_21h03.35.215
/Users/al

## B. Exclusion of participants who didn't answer short answers

In [149]:
# not present in SA analyses
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/73_1_online_exp_2020-06-10_14h37.05.252']
## 5ed1443918f6bd07449d88a5 - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b2/30/199_1_online_exp_2020-06-24_20h41.05.680']
# not present in SA analyses
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b3/45/345_1_online_exp_2020-07-16_09h07.09.087']
# not present in SA analyses
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/39_1_online_exp_2020-10-20_13h05.53.446']
# not present in SA analyses
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/31_1_online_exp_2020-10-20_11h38.47.446']

# 5/2/21 
## 5f5f9b5448f99d01c3b3101e - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/163_1_online_exp_2020-10-20_15h01.21.014']
## 5f970bac28901a06b8152b0b - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b7/54/47_1_online_exp_2020-11-06_11h52.08.207']
## 5f0e1c64a3a86a136d884628 - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/90_1_online_exp_2020-09-28_15h33.14.367']
## 5ced335ede266200161ea979 - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822']
## 5f70ae7ad1dae5090b80dbd9 - removed in SA exclusions
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/4_1_online_exp_2020-09-28_06h46.56.248']

e2 = 10

In [150]:
data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/4_1_online_exp_2020-09-28_06h46.56.248']

KeyError: '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/4_1_online_exp_2020-09-28_06h46.56.248'

In [151]:
for key in data_dict:
    if data_dict[key]['prolific_id'].iloc[0] == '5f70ae7ad1dae5090b80dbd9':
        print(key)



IndexError: single positional indexer is out-of-bounds

## C. Exclusion of participant with no data

In [152]:
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/6_1_online_exp_2020-10-20_10h04.26.275']

e3 = 1

## D. Exclusion of participants who said said more than half the sentences were a boundary

In [153]:
keys_to_del = []

for key in data_dict:
    kp = data_dict[key]['story_presses']
    N_kp= np.sum(kp == 9)
    this_story = data_dict[key]['story'].iloc[0]
    half_story_len = np.rint(len(story_boundaries[this_story])/2)
    if N_kp >= half_story_len:
        print(key)
        print(data_dict[key]['prolific_id'].iloc[0])
        keys_to_del.append(key)
        
for i in range(0, len(keys_to_del)):
    del data_dict[keys_to_del[i]]
    
e4 = len(keys_to_del)
new_n = len(data_dict)

/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot2/8/PARTICIPANT_1_online_exp_2020-05-11_20h42.51.597
5ce5b9844cd8130019acba32
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot4/7/PARTICIPANT_1_online_exp_2020-05-27_21h05.22.618
5d46c556fb0ce0001a04fd01
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/24_1_online_exp_2020-06-09_20h39.35.048
5ea616a090d52b61615dfb3b
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/21_1_online_exp_2020-06-06_15h20.03.392
5eda9f7c460b766ded73bdb7
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_p/10/18_1_online_exp_2020-06-06_14h58.27.858
571ceeb64ca277000953d1c1
/Users/alexreblando/Documents/Documents - Alex’s Mac

## E. Summary

In [5]:
print('The starting number of data is ', og_n)
print('The number of exclusions due to only one type of keypress is ', e1)
print('The number of exclusions due to not answering the short answers is ', e2)
print('The number of exclusions due to the participant have no useful data is ', e3)
print('The number of exclusions due to participant saying more than half the sentences are event boundaries ', e4)
print('The usable number of data is ', new_n)

The starting number of data is  391


NameError: name 'e1' is not defined

In [8]:
data_dict.keys()

dict_keys(['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h54.27.962', '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-06_20h00.18.643', '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h05.05.369', '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-09_01h01.48.508', '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online_exp_2020-05-07_23h40.02.268', '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot1/7/PARTICIPANT_1_online

## Exclusions for Recall

In [32]:
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/6_1_online_exp_2020-10-20_10h04.26.275']

In [5]:
from gensim.models import KeyedVectors #for word embeddings

wikipath = '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/analysis/free_recall/rolando/wiki-news-300d-1M.vec'
wv_model = KeyedVectors.load_word2vec_format(wikipath)

In [6]:
wv_dim = 300

# FastText preprocessing, based on bittlingmayer/ft_wiki_preproc.py
# Remove special characters, put spaces between all tokens
SUB = ["s/’/'/g", "s/′/'/g", "s/''/ /g", "s/'/ ' /g", 's/“/"/g', 's/”/"/g', 's/"/ /g', "s/\\./ \\. /g", "s/<br \\/>/ /g", "s/, / , /g", "s/(/ ( /g", "s/)/ ) /g", "s/\\!/ \\! /g", "s/\\?/ \\? /g", "s/\\;/ /g", "s/\\:/ /g", "s/-/ - /g", "s/=/ /g", "s/=/ /g", "s/*/ /g", "s/|/ /g", "s/«/ /g", 
       "s/…/ /g", "s/‘/ /g", "s/í/ /g", "s/ñ/ /g", "s/\x84/ /g", "s/î/ /g", "s/ó/ /g", "s/\x83/ /g", "s/ï/ /g", "s/õ/ /g",
       "s/ò/ /g", "s/,/ /g", "s/ô/ /g", "s/\x92/ /g", "s/é/ /g", "s/\x8e/ /g", "s/â\x80¦/ /g", "s/\x91/ /g", "s/\x93/ /g",
       "s/\x94/ /g", "s/ã®/ /g", "s/ã¨/ /g", "s/ã©/ /g",
       "s/\â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x9d/ /g", "s/â\x80\x99/ /g", "s/â\x80\x9c/ /g", "s/â\x80\x98/ /g",
       "s/â/ /g"]

def __normalize_text(s):
    for sg in SUB:
        rep = sg.replace('\\','').split('/')
        s = s.replace(rep[1], rep[2])
    s = s.replace('/',' ')
    return s

def __spaces(s):
    return ' '.join(s.split())

def __digits(s):
    return ''.join(filter(lambda c: not c.isdigit(), s))

# def preproc(s):
#     return __punctuation(__spaces(__digits(__normalize_text(s.lower()))))

def preproc(s):
    return (__spaces(__digits(__normalize_text(s.lower()))))

def __punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

def word2vecSent(sentence, model = 'fasttext'):
    wv_dim = 300 #for glove and fasttext
    
    if model == 'glove':
        wvmodel = glove_model
    elif model == 'fasttext':
        wvmodel = wv_model
        
    words = preproc(sentence).split(' ')
    wv = np.zeros((len(words), wv_dim))
    for i in range(len(words)):
        if words[i] in wvmodel.vocab:
            wv[i,:] = wvmodel.word_vec(words[i])
    
    return wv, len(words)

In [25]:
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/73_1_online_exp_2020-06-10_14h37.05.252']
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/102_1_online_exp_2020-09-28_15h37.58.717']
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/372_1_online_exp_2020-09-29_12h02.31.088']
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/39_1_online_exp_2020-10-20_13h05.53.446']
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/163_1_online_exp_2020-10-20_15h01.21.014']

In [26]:
del data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/163_1_online_exp_2020-10-20_15h01.21.014']

KeyError: '/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/163_1_online_exp_2020-10-20_15h01.21.014'

In [20]:
data_dict['/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b6/76/163_1_online_exp_2020-10-20_15h01.21.014']

,story_presses,A1,A2,A3,A4,A5,A6,A7,A8,distractor1,...,distractor3,distractor4,distractor5,prolific_id,story,p_type,persp,alt_persp,sa_order,recall
0,1,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
1,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
2,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
3,1,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
4,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
5,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
6,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
7,9,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
8,1,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this
9,1,""" 9 ""","""4""","""3""","""4""",""" ""","""3""","""34""","""4""",""" continue """,...,"""honest""",Shift,0,5f5f9b5448f99d01c3b3101e,42,2,40,2,1,i am really happy doing this


In [33]:
keys_to_del = []

for key in data_dict.keys():
    if isinstance(data_dict[key]['recall'].iloc[0], float):
        print('nan')
        keys_to_del.append(key)
    else:
        wv, lens = word2vecSent(data_dict[key]['recall'].iloc[0])
        if lens < 10:
            print('\n\n', key)
            print(data_dict[key]['recall'].iloc[0])
            
for i in range(0, len(keys_to_del)):
    del data_dict[keys_to_del[i]]



 /Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/177_1_online_exp_2020-06-10_14h45.30.620
Calvin intends to break up with his patner. 


 /Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/73_1_online_exp_2020-06-10_14h37.05.252
yes


 /Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/13_1_online_exp_2020-09-28_15h40.26.822
client arrive at the airports
nan
nan


 /Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/4_1_online_exp_2020-09-28_06h46.56.248
About a girl and her friend. 


 /Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/102_1_online_exp_2020-09-28_15h37.58.717
cannot describe it in details man.


 /Users/al

## F. Pickling data_dict for further use

In [28]:
pickle.dump( data_dict, open( 'usable_data_recall2', "wb" ) )

## E. Make summary of data and export it

In [29]:
loc_keys_r = dict()
soc_keys_r = dict()
none_keys_r = dict()
all_keys_r = dict()

for key in data_dict:
    this_story = data_dict[key]['story'].iloc[0]
    keys = data_dict[key]['story_presses']
    p_type = data_dict[key]['p_type'].iloc[0]
    #print out the filenames and prolific ids of people who did not press "9" once
    occurrences = np.count_nonzero(keys == 9)
    if occurrences == 0:
        print(key)
        print(data_dict[key]['prolific_id'].iloc[0])
    #put all story keys in the same dictionary regardless of priming
    if this_story in all_keys_r:
        all_keys_r[this_story] = np.concatenate((all_keys_r[this_story],keys[:,np.newaxis]), axis=1)
    else:
        all_keys_r[this_story] = keys[:, np.newaxis]
    #organize keys by priming type
    #no prime
    if p_type == 0:
        if this_story in none_keys_r:
            none_keys_r[this_story] = np.concatenate((none_keys_r[this_story],keys[:,np.newaxis]), axis=1)
        else:
            none_keys_r[this_story] = keys[:, np.newaxis]
    #loc prime
    elif p_type == 1:
        if this_story in loc_keys_r:
            loc_keys_r[this_story] = np.concatenate((loc_keys_r[this_story],keys[:,np.newaxis]), axis=1)
        else:
            loc_keys_r[this_story] = keys[:, np.newaxis]
    #soc prime
    elif p_type == 2:
        if this_story in soc_keys_r:
            soc_keys_r[this_story] = np.concatenate((soc_keys_r[this_story],keys[:,np.newaxis]), axis=1)
        else:
            soc_keys_r[this_story] = keys[:, np.newaxis]
            
sums = {}

for key in all_keys_r:
    this_sum = []
    x, y = all_keys_r[key].shape
    this_sum.append(y)
    if key in none_keys_r:
        x, y = none_keys_r[key].shape
        this_sum.append(y)
    else:
        this_sum.append(0)
    if key in loc_keys_r:
        x, y = loc_keys_r[key].shape
        this_sum.append(y)
    else:
        this_sum.append(0)
    if key in soc_keys_r:
        x, y = soc_keys_r[key].shape
        this_sum.append(y)
    else:
        this_sum.append(0)
    sums[key] = this_sum
    
pickle.dump( sums, open( 'sums_usable_data_recall2', "wb" ) )

/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/pilot2/8/PARTICIPANT_1_online_exp_2020-05-11_12h52.50.131
5cf0a300b094d700183e4d54
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b1/33/171_1_online_exp_2020-06-10_21h32.27.534
5de12fbfe4f42a1842172d7a
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/381_1_online_exp_2020-09-29_12h22.28.997
5f56056a645a6e9dba848fa2
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/394_1_online_exp_2020-09-29_12h55.24.161
5f58c77c4c2d8e0556d59fc1
/Users/alexreblando/Documents/Documents - Alex’s MacBook Air/GitHub/data_analysis_ebs/online_experiment/data/finalver_b5/78/373_1_online_exp_2020-09-29_12h00.12.404
5c2b4b959f18a9000179a141
/Users/alexreblando/Documents/Documents - Alex’s

<ipython-input-29-6654c42e02ac>:19: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

<ipython-input-29-6654c42e02ac>:26: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

<ipython-input-29-6654c42e02ac>:19: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

<ipython-input-29-6654c42e02ac>:26: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

<ipython-input-29-6654c42e02ac>:19: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is depr

In [ ]:
data